In [1]:
import gc
from tqdm.notebook import tqdm
import scipy
import scipy.sparse

import os
import pandas as pd
import scipy.sparse
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import pickle
import bz2
import shutil
from bz2 import BZ2Compressor
import random
import torch

import torch.nn as nn
import torch.utils.data as torch_data
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence

from torch.utils.tensorboard import SummaryWriter

import shutil

from file_utils import saver_state, loader_state, shuffle, compressed_read, compressed_write, decompress

import torch
import torch.nn as nn
from torch.nn.utils.rnn import PackedSequence
from typing import *

from models import LM, Net

In [2]:
#Do hard drive-friendly read and write?
do_compression = False

In [3]:
#Delte old data if there is. Recreate data directories.

path = os.path.join("nn_data","train")
if os.path.isdir(path):
    shutil.rmtree(path)
os.mkdir(os.path.join("nn_data","train"))

path = os.path.join("nn_data","dev")
if os.path.isdir(path):
    shutil.rmtree(path)
os.mkdir(os.path.join("nn_data","dev"))

path = os.path.join("nn_data","train_lm")
if os.path.isdir(path):
    shutil.rmtree(path)
os.mkdir(os.path.join("nn_data","train_lm"))

In [4]:
#copy data from read only directory to work directory
for i in range(12):
    shutil.copy(os.path.join("nn_data","clear","chunk"+str(i)+".bz2"), os.path.join("nn_data","train"))

for i in range(12):
    shutil.copy(os.path.join("nn_data","clear","chunk"+str(i)+".bz2"), os.path.join("nn_data","train_lm"))

In [5]:
#if not flag than do not perform compression due to speed reasons            
if(do_compression == False):
    decompress("train")
    decompress("train_lm")

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [6]:
#make classifier validation
shutil.move(os.path.join("nn_data","train","chunk"+str(10)+".bz2"), os.path.join("nn_data","dev","chunk"+str(10)+".bz2"))
shutil.move(os.path.join("nn_data","train","chunk"+str(11)+".bz2"), os.path.join("nn_data","dev","chunk"+str(11)+".bz2"))

'nn_data\\dev\\chunk11.bz2'

In [7]:
shuffle()
shuffle("train_lm")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [3]:
#load categorical to ids mapping
with open('feat2id.pickle', 'rb') as handle:
    feat2id = pickle.load(handle)

with open('id2feat.pickle', 'rb') as handle:
    id2feat = pickle.load(handle)

with open('global_mapping.pickle', 'rb') as handle:
    global_mapping = pickle.load(handle)

In [4]:
pad_values = {}
for ft in feat2id.keys():
    MaxFeatureRange = len(global_mapping[ft])+1+1+1 #reserve space for special tokens
    pad_values[ft] = MaxFeatureRange

In [5]:
#next example generator. Read files from chunk to chunk and yiel exaples from each chunk

import string
latters = list(string.ascii_uppercase)+list(string.ascii_lowercase)
latters2id = {k:v for k,v in zip(latters, range(len(latters)))}
id2latters = {v:k for k,v in latters2id.items()}

def get_next_example(part = "train", avaliable_chunks=(0,1,2,3,4,5,6,7,8,9)):
    for chunk_i in avaliable_chunks:
        with open(os.path.join("nn_data",part,"chunk"+str(chunk_i)+".bz2"), "rb") as f:
            lines = compressed_read(f, do_compression = do_compression)
        lines = lines.split('\n')
        for l in lines:
            data = l.split(' ')
            words = data[:-1]
            y = data[-1]
            
            list_of_tokens_lists = [[latters2id[latter] for latter in word] for word in words]
            
            list_of_ids_lists = []
            padding_shift = 3
            
            for tokl, nom in zip(list_of_tokens_lists, range(len(list_of_tokens_lists)) ):
                list_of_ids_lists.append([global_mapping[id2feat[nom]][x] + padding_shift for x in tokl])
            yield list_of_ids_lists + [int(y)]

In [6]:
#data convertion functions for autoencoder
max_pos_len = 50
def process_feature_lm(l, max_pos_len):
    if(len(l) < max_pos_len):
        return ([1] + l + [2] + [0]*(max_pos_len - len(l)), l + [2] + [0]*(max_pos_len - len(l)+1))
    elif(len(l) > max_pos_len):
        return ([1] + l[-max_pos_len:] + [2], l[-max_pos_len-1:] + [2])#[::-1]
    else:
        return ([1] + l + [2], l + [2] + [0])

#gather examples into batch   
def collate_fn_lm(batch_list, max_pos_len):
    Xs = [[] for _ in range(len(id2feat))]
    Ys = [[] for _ in range(len(id2feat))]
    lens = []
    lens_tech = []
    
    for example in batch_list:
        l = example[0]
        lens_tech.append(len(l) if(len(l) < max_pos_len) else max_pos_len)

    for example in batch_list:
        for i in range(len(id2feat)):
            l = example[i]
            if(i==0):
                lens.append(len(l) + 2 if(len(l) < max_pos_len) else max_pos_len+2)
                
            x, y  = process_feature_lm(l, min(max_pos_len, max(lens_tech)))
            Xs[i].append(x)
            Ys[i].append(y)
            #print(len(x), len(y))
    
    return ([torch.tensor(x, dtype=torch.int32) for x in Xs], [torch.tensor(y, dtype=torch.int64) for y in Ys], lens)

#generate batches for autoencoder
def batch_gen_lm(batch_size, max_pos_len = 50, part = "train_lm", avaliable_chunks=(0,1,2,3,4,5,6,7,8,9,10,11)):
    batch_list = []
    i = 0
    for x in get_next_example(part, avaliable_chunks):
        batch_list.append(x)
        i += 1
        if(i == batch_size):
            yield collate_fn_lm(batch_list, max_pos_len)
            batch_list = []
            i = 0
    if(len(batch_list)>0):
        yield collate_fn_lm(batch_list)

In [7]:
#data convertion functions for classifier
def process_feature(l, max_pos_len):
    if(len(l) < max_pos_len):
        l = [1] + l  + [0]*(max_pos_len - len(l))
    elif(len(l) > max_pos_len):
        l = [1] + l[-max_pos_len:] #[::-1]
    else:
        l = [1] + l  #[::-1]
    return l

#gather examples into batch
def collate_fn(batch_list, max_pos_len):
    feat_lists = [[] for _ in range(len(id2feat))]
    y = []
    lens = []
    for example in batch_list:
        for i in range(len(id2feat)):
            l = example[i]
            if(i==0):
                lens.append(len(l) + 1 if(len(l) < max_pos_len) else max_pos_len+1)
            l  = process_feature(l, max_pos_len)
            feat_lists[i].append(l)
        y.append(example[-1])
    
    return ([torch.tensor(x, dtype=torch.int32) for x in feat_lists], torch.tensor(y, dtype=torch.int64), lens)

#generate batches for classifier
def batch_gen(batch_size, max_pos_len = 50, part = "train", avaliable_chunks=(0,1,2,3,4,5,6,7,8,9)):
    batch_list = []
    i = 0
    for x in get_next_example(part, avaliable_chunks):
        batch_list.append(x)
        i += 1
        if(i == batch_size):
            yield collate_fn(batch_list, max_pos_len)
            batch_list = []
            i = 0
    if(len(batch_list)>0):
        yield collate_fn(batch_list)


In [8]:
model = LM(id2feat, pad_values)
classifier_model = Net(id2feat, pad_values)

embedding size [18, 16, 16, 16, 16, 16, 18, 10, 8, 4, 6, 14, 16, 18, 12, 8, 12, 4, 4, 4, 4, 4, 18, 18, 18, 4, 4, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 10, 10, 10, 6, 4, 4]
embedding size [18, 16, 16, 16, 16, 16, 18, 10, 8, 4, 6, 14, 16, 18, 12, 8, 12, 4, 4, 4, 4, 4, 18, 18, 18, 4, 4, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 10, 10, 10, 6, 4, 4]


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
model.to(device)
optimizer_lm = optim.Adam(model.parameters(), lr=1e-3)
criterion_lm = nn.NLLLoss(ignore_index=0)

In [11]:
#classifier evaluation
def eval_dev_classifier(model, nbatch):
    targets = []
    preds = []
    ii = 0
    for X, y, lens in tqdm(batch_gen(1000, max_pos_len, "dev", (10,11))):
        for i in range(len(X)):
            X[i] = X[i].to(device)
            y[i] = y[i].to(device)
        tgt = y.numpy()
        output = model(X, lens)[:,1] 
        pr = output.detach().cpu().numpy()
        targets.append(tgt)
        preds.append(np.exp(pr))
        ii += 1
        
        if(ii>nbatch):
            break
    return roc_auc_score(np.hstack(targets), np.hstack(preds))

In [12]:
from torch.optim.lr_scheduler import ExponentialLR

In [13]:
#classifier training
def train_classifier(epoches_gone, experiment_name, device):
    load_dir = os.path.join(experiment_name, "lm_chekpoints", str(epoches_gone), "model.pt")
    model = loader_state(load_dir)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    scheduler = ExponentialLR(optimizer, gamma = (1e-5/1e-3)**(1/(6000*7)))

    loss_function = torch.nn.NLLLoss(weight=torch.tensor([1., 0.97/0.03]).to(device))#0.97/0.03
    
    logdir=os.path.join(experiment_name, "classifier_logs", str(epoches_gone))
    
    if os.path.isdir(logdir):
        shutil.rmtree(logdir)

    os.mkdir(logdir)

    writer=SummaryWriter(log_dir=logdir)
    
    
    model.train()
    epoches=3
    ii = 0
    for ep in range(epoches):
        for X, y, lens in tqdm(batch_gen(500, max_pos_len)):
            for i in range(len(X)):
                X[i] = X[i].to(device)
            y = y.to(device)
            output = model(X, lens)
            loss = loss_function(output, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()
            writer.add_scalar('Loss/train', loss.item() , ii)
            if(ii%100 == 0):
                print(loss.item())
            if(ii%500 == 0):
                model.eval()
                ev = eval_dev_classifier(model, 50)
                print("after",ii,"steps short auc dev", ev)
                writer.add_scalar('auc dev short', ev , ii)
                model.train()
            ii+=1
        #break
        model.eval()
        ev = eval_dev_classifier(model, 10**4)
        print("after",ep,"epoches full auc dev", ev)
        writer.add_scalar('auc dev long', ev , ii)
        model.train()
        shuffle("train")
    

In [14]:
experiment_name = 'with_test_nn2'

In [15]:
#manage log directories
if os.path.isdir(experiment_name):
    shutil.rmtree(experiment_name)

os.mkdir(experiment_name)
os.mkdir(os.path.join(experiment_name, "lm_chekpoints"))
os.mkdir(os.path.join(experiment_name, "classifier_logs"))

In [16]:
#autoencoder pretraining 
#run 5 epoch. I pretrain model on all avaliable data. Save state after each epoch
model.train()
epoches=1
ii = 0
for ep in range(epoches):
    for X, y, lens in tqdm(batch_gen_lm(1000, max_pos_len)):
        for i in range(len(X)):
            X[i] = X[i].to(device)
            y[i] = y[i].to(device)
        output = model(X, lens)
        ct = []
        for i in range(len(X)):
            ct.append(criterion_lm(output[i], y[i]))
            
        loss = torch.stack(ct).mean()
        loss.backward()
        optimizer_lm.step()
        optimizer_lm.zero_grad()
        
        if((ii+1)%100 == 0):
            print(loss.item())
        ii+=1
        
    os.mkdir(os.path.join(experiment_name, "lm_chekpoints", str(ep)))
    
    saver_state(model, os.path.join(experiment_name, "lm_chekpoints", str(ep), "model.pt"))

    shuffle("train_lm")

0it [00:00, ?it/s]

D:\anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\rnn.py:815: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\RNN.cpp:982.)
  result = _VF.lstm(input, batch_sizes, hx, self._flat_weights, self.bias,
D:\anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\rnn.py:812: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\RNN.cpp:982.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


0.8712790608406067
0.7818545699119568
0.7233870029449463
0.6848558187484741
0.7006369233131409
0.6945775151252747
0.657108724117279
0.6356131434440613
0.6248583793640137
0.6150219440460205
0.6237711906433105
0.6269310712814331
0.6181519031524658
0.5937729477882385
0.579681396484375
0.587653636932373
0.5703003406524658
0.591055154800415
0.572417676448822
0.5812262296676636
0.5671764612197876
0.552343487739563
0.566238522529602
0.5516991019248962


In [28]:
del model
del optimizer_lm
del X
del y

In [36]:
#load weights from each checkpoint and check classifier's performance
for ep in range(5):
    train_classifier(ep, experiment_name, device)

embedding size [9, 8, 8, 8, 8, 8, 9, 5, 4, 2, 3, 7, 7, 9, 6, 4, 5, 2, 2, 2, 2, 2, 9, 9, 9, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 3, 2, 2]


0it [00:00, ?it/s]

1.0004462003707886


0it [00:00, ?it/s]

after 0 steps short auc dev 0.48258688902005115
0.6192314028739929
0.5748345851898193
0.6302871704101562
0.5617143511772156
0.5921562910079956


...

0it [00:00, ?it/s]

after 1500 steps short auc dev 0.7304860883736771
0.5615350008010864
0.6070448160171509
0.6122964024543762
0.5485662817955017
0.5461611151695251


0it [00:00, ?it/s]

after 2000 steps short auc dev 0.7364831906581121


KeyboardInterrupt: 